In [ ]:
#%pip install semantic-link

In [ ]:
import sempy.fabric as fabric
from datetime import datetime, timedelta
from pyspark.sql.functions import *

In [ ]:
number_of_days = 15
fabric_capacity_workspace = "354b300f-3c0f-4f01-b3c2-03a0c06624aa"
direct_query_semantic_model = "Fabric_Capacity_Direct_Query"

In [ ]:
df_capacities = fabric.evaluate_dax(dataset = direct_query_semantic_model, workspace= fabric_capacity_workspace, dax_string = "EVALUATE DISTINCT( 'Capacities'[capacityID] )")

In [ ]:
df_capacities.columns = ["capacityId"]
df_capacities

In [ ]:
now = datetime.now()

# List to hold the last x days
last_x_days = []

# Generate the last x days
for i in range(number_of_days):
    day = now - timedelta(days=i)
    last_x_days.append(day.strftime('%Y-%m-%d'))

In [ ]:
df_capacities = df_capacities
for day in last_x_days:
    print(day)
    for index, row in df_capacities.iterrows():
        cap_id = (row["capacityId"])
        print(cap_id)
        dax_string = """DEFINE
                MPARAMETER CapacityID = "{cap_id}"

                VAR __DS0FilterTable = 
                    TREATAS({{DATEVALUE("{day}")}}, Dates[Day])

                VAR __DS0FilterTable2 = 
                    TREATAS({{"{cap_id}"}}, Capacities[capacityId])

                VAR __DS0Core = 
                FILTER(
                        KEEPFILTERS(
                    SUMMARIZECOLUMNS(
                        Capacities[capacityId],
                        Items[WorkspaceName],
                        Items[ItemName],
                        MetricsByItemandOperationandDay[OperationName],
                        Capacities[Capacity Name],
                        __DS0FilterTable,
                        "Duration", CALCULATE(SUM(MetricsByItemandOperationandDay[sum_duration])),
                        "Count_Users", CALCULATE(SUM(MetricsByItemandOperationandDay[count_users])),
                        "CU", CALCULATE(SUM(MetricsByItemandOperationandDay[sum_CU])),
                        "Count_operations", CALCULATE(SUM(MetricsByItemandOperationandDay[count_operations])),
                        "DurationMS", CALCULATE(SUM(MetricsByItemandOperationandDay[avg_DurationMS]))
                    )	),
                        [CU] <> 0
                    )

                VAR __DS0PrimaryWindowed = 
                    TOPN(
                        501,
                        __DS0Core,
                        Capacities[Capacity Name],
                        1,
                        Capacities[capacityId],
                        1,
                        Items[WorkspaceName],
                        1,
                        Items[ItemName],
                        1,
                        MetricsByItemandOperationandDay[OperationName],
                        1
                    )

            EVALUATE
                __DS0PrimaryWindowed

            ORDER BY
                Capacities[Capacity Name],
                Capacities[capacityId],
                Items[WorkspaceName],
                Items[ItemName],
                MetricsByItemandOperationandDay[OperationName] """
        df = fabric.evaluate_dax(dataset = direct_query_semantic_model, workspace= fabric_capacity_workspace, dax_string = dax_string.format( cap_id = cap_id, day = day )) 
        df.columns = ['CapacityId', 'WorkspaceName', 'ItemName', 'OperationName', 'CapacityName', 'Duration', 'Count_Users', 'CU','Count_operations', 'DurationMS']
        if not(df.empty):
            df = spark.createDataFrame(df)
            df = df.withColumn("Date", lit(day))
            if spark.catalog.tableExists("Metrics_History"):
                spark.sql("DELETE FROM Metrics_History WHERE Date ='" + day + "' and CapacityId = '" + cap_id + "'")
            df.write.mode("append").option("mergeSchema", "true").format("delta").save("Tables/Metrics_History")
